In [ ]:
# TODO: Refactor and parallelize.

import requests
from bs4 import BeautifulSoup
import re

last = "0000004" # Makes you wonder that happend with the 4 first words.
final_id = "3709394" # Get manually.

headers = {
    "Accept": "text/html, */*; q=0.01",
    "Accept-Language": "sv-SE,sv;q=0.9",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://svenska.se",
    "Referer": "https://svenska.se/saol/?sok=b&pz=1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
}

url = "https://svenska.se/wp-admin/admin-ajax.php"

# Fetch all words without conjugations.

with open("saol_out_0.txt", "w", encoding="UTF_8") as f:
    while last != final_id:
        payload = {
            "action": "myprefix_scrollist",
            "unik": last,
            "dir": "ned",
            "dict": "saol",
        }

        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        hfrefs_n_words = soup.find_all("a", class_="slank")

        if hfrefs_n_words:
            for href_n_word in hfrefs_n_words:
                word = href_n_word.find(
                    "span", class_="plain").text.strip()
                href = href_n_word["href"]
                f.write(f"{word=} : {href=} \n")

            last_href = hfrefs_n_words[-1]["href"]
            match = re.search(r"id=(\d+)\D", last_href)
            if match:
                last = match.group(1)

In [ ]:
# Remove words containing illegal characters.

saol = open("saol_out_0.txt", encoding="utf-8").readlines()
lista = ['i', 'o', 'å', 'ä']

with open("saol_out_1.txt", "w", encoding="UTF_8") as f:
    prev = ""
    for i in range(len(saol)):
        word = saol[i].split(' : ')[0].replace("word='", '').replace("'", '')
        if word == prev or (len(word) == 1 and word not in lista):
            continue
#         if re.fullmatch(r'[a-zA-ZåäöÅÄÖ]+', word) is None:
        if re.fullmatch(r'^(?=.*[A-ZÅÄÖ])[a-zA-ZåäöÅÄÖ]+$', word) is None:
            continue
        f.write(saol[i])
        prev = word

In [ ]:
saol = open("saol_out_1.txt", encoding="UTF_8-8").readlines()

with open("saol_out_2.txt", "w", encoding="UTF_8") as f:
    for i in saol:      
        url = i.split("href='")[1].replace("'", '')
        response = requests.post(url, headers=headers)
        
        try:
            response.raise_for_status()

            soup = BeautifulSoup(response.text, "html.parser")

            asd = soup.find_all("tr")

            klass = soup.find_all(class_='ordklass')
            if not klass:
                continue

            klass = str(klass[0])
            f.write("klasstyp " + klass.replace('<a class="ordklass">', '').replace('</a>', '') + '\n')

            for i in asd[1:]:
                i = str(i)
                if 'class="ordformth"' in i:
                    continue

                f.write(i.split('class="bform">')[1].split('</span></b>')[0] + '\n')
        except:
            print('invalid', url)

In [ ]:
saol = open("saol_out_2.txt", encoding="UTF_8-8").readlines()

with open("saol_out_3.txt", "w", encoding="UTF_8") as f:
    b = False
    for i in saol:
        if 'klasstyp' in i:
            b = False
        if i == 'klasstyp namn':
            b = True
        if b:
            continue
        i = i.lower()
        if 'klasstyp' not in i and re.fullmatch(r'[a-zåäö]+', word):
            f.write(f'{i.upper()}')